https://huggingface.co/yangheng/deberta-v3-large-absa-v1.1

# Imports

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification, pipeline

from lib.sentiment_analysis_utils import combine_lede_and_text, remove_text_formatting, read_all_news_in_dir

import os

In [ ]:
#import torch

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.cuda.is_available()

# Load and preprocess data

In [ ]:
df_en_raw = read_all_news_in_dir(os.getcwd() + "/../data_preparation/raw_data/en/")
df_en_raw

The above method iterates over files in specified directory, reading and combining the content into single dataframe.

In [ ]:
df_en_raw = combine_lede_and_text(df_en_raw)
df_en_raw = remove_text_formatting(df_en_raw)
df_en_raw

The above methods do 2 tasks.

First method combines lede with the rest of the article text. Data from STA is organized in such a way that first paragraph of the article is separated from the rest of the text which is usually displayed only under paid subscription (first paragraph for free). We want to analyze both, therefore we combine.

Second method is responsible for removing observed by us text formatting attributes (i.e. we want to remove \n\n that separates article paragraphs, or html formatting tags like <b> </b> used to display test in bold).

# Classify polarity

In [ ]:
df_en_raw = df_en_raw.head(10)

As this is just PoC of the project, we limit the number of articles to reduce the time for computations. We want to do some testing on smaller batch of data before we test our solution on something computationally expensive.

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")
# model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")

sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

In [ ]:
for i, row in df_en_raw.iterrows():
    df_en_raw.loc[i, 'overall_sentiment'] = sentiment_analysis(' '.join(row.whole_text.split(' ')[:350]))


In [ ]:
df_en_raw

In [ ]:
df_en_raw.to_csv('example_results.csv')

Results comment: model seems to provide something reasonable, however, we are still in the process of creating labelled dataset, therefore we will put bigger attention to the evaluation of models in the following weeks. Here is just PoC, we will validate performance in more detail.

We have checked manually a few examples (that's why we consider results reasonable), however, we are unable to disclose these sample news articles in public repository.

Moreover, we put a hard limit on the number of words in articles (we trim it arbitrary), to fit it to the input of the model that is max 512 token long. (we trim to 350 as result from tokenizer is sometimes longer than number of words). To solve it, we will probably also experiment with 2 approaches: 1. changing the model to one that accepts input of sizes like 4096 tokens 2. summarizing the article with another model before passing for sentiment analysis.